In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import shutil

In [ ]:
# Import Library
import os
import shutil
import glob
import random

# Definisi Path dan Kelas Makanan
SOURCE_BASE_PATH = "./dataset_gambar_unzip"

DEST_BASE_PATH = "./dataset_gambar"

# Daftar 13 kelas
CLASSES = ["Ayam Goreng", "Burger", "French Fries", "Gado-Gado", "Ikan Goreng", "Mie Goreng", "Nasi Goreng", "Nasi Padang", "Pizza", "Rawon", "Rendang", "Sate", "Soto"]

# Rasio pembagian dataset
TRAIN_RATIO = 0.70
VALIDATION_RATIO = 0.15
TEST_RATIO = 0.15

# Membuat Struktur Direktori Baru dan Memisahkan Data
def create_and_split_dataset():

    # Hapus direktori lama jika ada untuk memulai dari awal
    if os.path.exists(DEST_BASE_PATH):
        shutil.rmtree(DEST_BASE_PATH)

    print(f"Membuat struktur direktori di: {DEST_BASE_PATH}")

    for split in ["train", "validation", "test"]:
        for food_class in CLASSES:
            path = os.path.join(DEST_BASE_PATH, split, food_class)
            os.makedirs(path, exist_ok=True)

    print("Struktur direktori berhasil dibuat.")
    print("-" * 30)

    for food_class in CLASSES:
        source_dir = os.path.join(SOURCE_BASE_PATH, food_class)
        all_images = glob.glob(os.path.join(source_dir, '*.jpg')) + \
                     glob.glob(os.path.join(source_dir, '*.png')) + \
                     glob.glob(os.path.join(source_dir, '*.jpeg'))

        random.shuffle(all_images)

        total_images = len(all_images)
        train_end = int(total_images * TRAIN_RATIO)
        validation_end = train_end + int(total_images * VALIDATION_RATIO)

        train_files = all_images[:train_end]
        validation_files = all_images[train_end:validation_end]
        test_files = all_images[validation_end:]

        # Fungsi untuk menyalin file
        def copy_files(files, split_name):
            for file_path in files:
                shutil.copy(file_path, os.path.join(DEST_BASE_PATH, split_name, food_class))

        # Menyalin file ke direktori masing-masing
        copy_files(train_files, "train")
        copy_files(validation_files, "validation")
        copy_files(test_files, "test")

        print(f"Kelas: {food_class}")
        print(f"  - Total gambar: {total_images}")
        print(f"  - Dialokasikan untuk Training: {len(train_files)}")
        print(f"  - Dialokasikan untuk Validasi: {len(validation_files)}")
        print(f"  - Dialokasikan untuk Testing: {len(test_files)}")
        print("-" * 30)

# Menjalankan fungsi
create_and_split_dataset()

Membuat struktur direktori di: ./dataset_gambar
Struktur direktori berhasil dibuat.
------------------------------
Kelas: Ayam Goreng
  - Total gambar: 0
  - Dialokasikan untuk Training: 0
  - Dialokasikan untuk Validasi: 0
  - Dialokasikan untuk Testing: 0
------------------------------
Kelas: Burger
  - Total gambar: 0
  - Dialokasikan untuk Training: 0
  - Dialokasikan untuk Validasi: 0
  - Dialokasikan untuk Testing: 0
------------------------------
Kelas: French Fries
  - Total gambar: 0
  - Dialokasikan untuk Training: 0
  - Dialokasikan untuk Validasi: 0
  - Dialokasikan untuk Testing: 0
------------------------------
Kelas: Gado-Gado
  - Total gambar: 0
  - Dialokasikan untuk Training: 0
  - Dialokasikan untuk Validasi: 0
  - Dialokasikan untuk Testing: 0
------------------------------
Kelas: Ikan Goreng
  - Total gambar: 0
  - Dialokasikan untuk Training: 0
  - Dialokasikan untuk Validasi: 0
  - Dialokasikan untuk Testing: 0
------------------------------
Kelas: Mie Goreng
  -

In [ ]:
# Impor TensorFlow dan Definisi Parameter
import tensorflow as tf

BATCH_SIZE = 32
IMG_SIZE = (256, 256)

# Memuat Dataset dari Direktori
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DEST_BASE_PATH, "train"),
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DEST_BASE_PATH, "validation"),
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DEST_BASE_PATH, "test"),
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE)

# Cell 7: Menampilkan Nama Kelas dan Contoh Bentuk Data
class_names = train_ds.class_names
print(f"Kelas yang ditemukan: {class_names}")

for images, labels in train_ds.take(1):
    print(f"Bentuk batch gambar: {images.shape}") # (Batch Size, Height, Width, Channels)
    print(f"Bentuk batch label: {labels.shape}")

Found 0 files belonging to 13 classes.


ValueError: No images found in directory ./dataset_gambar/train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
# Visualisasi Sampel Gambar dari Training Set
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for images, labels in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Sampel Gambar dari Training Set", fontsize=16)
plt.show()

# Konfigurasi Dataset untuk Performa
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

NameError: name 'train_ds' is not defined

<Figure size 1200x800 with 0 Axes>

In [ ]:
# Mendefinisikan Lapisan Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

# Membangun Arsitektur Model Lengkap
NUM_CLASSES = len(class_names)
INPUT_SHAPE = IMG_SIZE + (3,)

model = tf.keras.Sequential([
    # Input Layer
    tf.keras.layers.InputLayer(input_shape=INPUT_SHAPE),

    # Augmentasi data
    data_augmentation,

    # Normalisasi piksel
    tf.keras.layers.Rescaling(1./255),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5), # Mencegah overfitting
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax') # Output layer
])

# Menampilkan Ringkasan Arsitektur Model
model.summary()

NameError: name 'class_names' is not defined

In [ ]:
# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy", # Sesuai untuk label integer [cite: 126]
    metrics=["accuracy"]
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", # Memantau akurasi validasi
    patience=5,             # Berhenti setelah 5 epoch tanpa peningkatan
    restore_best_weights=True # Mengembalikan bobot terbaik yang ditemukan
)

# Model Training
EPOCHS = 50

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

In [ ]:
# Memvisualisasikan Hasil Pelatihan
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluasi Final pada Test Set
print("\n" + "="*30)
print("Mengevaluasi model dengan data Test...")
test_loss, test_accuracy = model.evaluate(test_ds)
print("="*30)
print(f"Akurasi pada Test Set: {test_accuracy*100:.2f}%")
print(f"Loss pada Test Set: {test_loss:.4f}")

In [ ]:
# Menyimpan Model ke File
MODEL_FILENAME = "food_classifier_v1.h5"
model.save(MODEL_FILENAME)
print(f"\nModel telah berhasil disimpan sebagai {MODEL_FILENAME}")

In [ ]:
# Setup Gradio
import gradio as gr
import numpy as np
import tensorflow as tf
import pandas as pd

# Load model
loaded_model = tf.keras.models.load_model(MODEL_FILENAME, compile=False)
print("Model berhasil dimuat.")

# Load dataset kalori
kalori_df = pd.read_excel("kalori.xlsx")
kalori_dict = dict(zip(kalori_df["makanan"].str.lower(), kalori_df["kalori"]))

# Prediksi
def predict_image(image):
    if image is None:
        return "Mohon unggah gambar terlebih dahulu."

    # Ubah ukuran gambar agar sesuai dengan input model
    img_resized = tf.image.resize(image, IMG_SIZE)
    img_array = tf.expand_dims(img_resized, 0)

    predictions = loaded_model.predict(img_array)

    # Menampilkan skor probabilitas menggunakan softmax
    scores = tf.nn.softmax(predictions[0])
    confidences = {}

    for i in range(len(class_names)):
        class_name = class_names[i]
        prob = float(scores[i])

        # Jika kelas ini adalah prediksi top-1, tambahkan info kalori
        if i == int(tf.argmax(scores)):
            kalori = kalori_dict.get(class_name.lower(), "Data kalori tidak tersedia")
            class_name = f"{class_name} ({kalori} kalori)"

        confidences[class_name] = prob

    return confidences

# 3. Konfigurasi dan luncurkan antarmuka Gradio
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Unggah Gambar Makanan Anda"),
    outputs=gr.Label(num_top_classes=3, label="Hasil Prediksi"),
    title="CalClick",
    description="Model ini dibangun dengan TensorFlow & Keras.",
    examples=[],
    allow_flagging="never" # Menonaktifkan tombol "Flag"
)

# Launch
interface.launch(debug=True, share=True) # set share=True untuk emndapatkan link publik